In [1]:
#https://www.youtube.com/watch?v=DmJXgWmq3pY&list=PL50mYnndduIHGS49Q_tve1f7aW4NHjvgQ&index=1
#Completed all the rdd related video 
#Still need to check the one with databricks in this playlist
#https://www.youtube.com/watch?v=2otrn2mvlSQ
#Use the above link and move to next tutorials

In [2]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

23/06/21 12:55:35 WARN Utils: Your hostname, laptop resolves to a loopback address: 127.0.0.1; using 192.168.0.9 instead (on interface wlp3s0)
23/06/21 12:55:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/21 12:55:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
matchDayResults_rdd = sc.textFile("/home/desai/DE/Files/pl_stats_2023/2023_matchday_results.csv")

In [4]:
lines = matchDayResults_rdd.take(5)
for line in lines:
    print(line)

Sl.no,fixture.id,fixture.date,teams.home.id,teams.home.name,teams.home.winner,teams.away.id,teams.away.name,teams.away.winner,goals.home,goals.away
0,867946,2022-08-05T19:00:00+00:00,52,Crystal Palace,False,42,Arsenal,True,0,2
1,867947,2022-08-06T11:30:00+00:00,36,Fulham,,40,Liverpool,,2,2
2,867948,2022-08-06T14:00:00+00:00,35,Bournemouth,True,66,Aston Villa,False,2,0
3,867949,2022-08-06T14:00:00+00:00,63,Leeds,True,39,Wolves,False,2,1


In [5]:
x= sc.parallelize(['a',1,3,1,3,'a','b'])
x.countByValue()

defaultdict(int, {'a': 2, 1: 2, 3: 2, 'b': 1})

In [6]:
#fold
from operator import add
numbers_rdd = sc.parallelize(range(1, 11), numSlices=3)
numbers_rdd.fold(0,add)

55

In [7]:
numbers_rdd = sc.parallelize(range(1, 11), numSlices=3)
print(numbers_rdd.glom().collect())
#print(numbers_rdd.collect())
# Define the binary operator for summation
def add(x, y):
    return x + y

# Perform fold operation on each partition to get sum of each partition
partition_sums = numbers_rdd.mapPartitions(lambda partition: [folded_result := 0] + [folded_result := add(folded_result, num) for num in partition])

# Collect the partition sums as a list
partition_sums_list = partition_sums.collect()

# Print the sum of each partition
for i, partition_sum in enumerate(partition_sums_list):
    print(f"Partition {i + 1} sum:", partition_sum)

[[1, 2, 3], [4, 5, 6], [7, 8, 9, 10]]
Partition 1 sum: 0
Partition 2 sum: 1
Partition 3 sum: 3
Partition 4 sum: 6
Partition 5 sum: 0
Partition 6 sum: 4
Partition 7 sum: 9
Partition 8 sum: 15
Partition 9 sum: 0
Partition 10 sum: 7
Partition 11 sum: 15
Partition 12 sum: 24
Partition 13 sum: 34


In [8]:
x_map = sc.parallelize(range(1,11))
y_map = x_map.map(lambda x:(x,x**2))
print('Values x_map:{0}'.format(x_map.collect()))
print('Values y_map:{0}'.format(y_map.collect()))

Values x_map:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Values y_map:[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25), (6, 36), (7, 49), (8, 64), (9, 81), (10, 100)]


In [9]:
x_filter = sc.parallelize(range(1,11))
y_filter = x_map.filter(lambda x:x%2 == 0)
print('Values x_filter:{0}'.format(x_filter.collect()))
print('Values y_filter:{0}'.format(y_filter.collect()))

Values x_filter:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Values y_filter:[2, 4, 6, 8, 10]


In [10]:
x_mapPartition = sc.parallelize([1,2,3,4,5,6],2)
def f(iterator): yield sum(iterator)
y_mapPartition= x_mapPartition.mapPartitions(f)
#glom() flattens elements of the same partition
print(x_mapPartition.glom().collect())
print(y_mapPartition.glom().collect())

[[1, 2, 3], [4, 5, 6]]
[[6], [15]]


In [11]:
#groupByKey
x_gb= sc.parallelize([('B',5),('B',4),('A',3),('A',2),('C',3)])
y_gb= x_gb.groupByKey()
print(y_gb.collect())
print([(j[0],[i for i in j[i]]) for j in y_gb.collect()])

[('B', <pyspark.resultiterable.ResultIterable object at 0x7f2115df4ee0>), ('A', <pyspark.resultiterable.ResultIterable object at 0x7f2115df4f40>), ('C', <pyspark.resultiterable.ResultIterable object at 0x7f2115df4e20>)]


IndexError: tuple index out of range

In [12]:
#Spark Caching and Persistence

In [13]:
matchDayResults_rdd = sc.textFile("/home/desai/DE/Files/pl_stats_2023/2023_matchday_results.csv")

In [16]:
import timeit
start=timeit.default_timer()
matchDayResults_rdd.count()
matchDayResults_rdd.min()
matchDayResults_rdd.max()
matchDayResults_rdd.collect()
end=timeit.default_timer()
print("elapsed time: {}".format(end-start))

elapsed time: 0.38904363699975875


In [17]:
matchDayResults_rdd.cache()

/home/desai/DE/Files/pl_stats_2023/2023_matchday_results.csv MapPartitionsRDD[26] at textFile at NativeMethodAccessorImpl.java:0

In [18]:
import timeit
start=timeit.default_timer()
matchDayResults_rdd.count()
matchDayResults_rdd.min()
matchDayResults_rdd.max()
matchDayResults_rdd.collect()
end=timeit.default_timer()
print("elapsed time: {}".format(end-start))

elapsed time: 0.3070466670001224


In [20]:
matchDayResults_rdd.unpersist()

/home/desai/DE/Files/pl_stats_2023/2023_matchday_results.csv MapPartitionsRDD[26] at textFile at NativeMethodAccessorImpl.java:0

In [23]:
import pyspark
matchDayResults_rdd.persist(pyspark.StorageLevel.DISK_ONLY)

/home/desai/DE/Files/pl_stats_2023/2023_matchday_results.csv MapPartitionsRDD[26] at textFile at NativeMethodAccessorImpl.java:0

In [24]:
#Broadcast Variables

In [26]:
#Joins 
#join
#leftOuterJoin
#rightOuterJoin
#cartesian

In [1]:
#difference between rdd & dataframe & dataset
#Test
#Test123